#### Optimization & Analytics 23/24

#### Names:

##### - Mohamed Afif Chifaoui (100452024)
##### - Ricardo Vazquez Alvarez (100417117)

---


# Network Optimization and Non-linear models HW2

## 1. Network problem

Our **network optimization** relies on the Max-Flow Problem.
Spain is known to be the **main** exporter in the world, with anual exportations surpassing the 210 thousand of tons.

However, in order to export these quantities the neighboring countries of Spain only allow a certain amount of oil to be passed through their roads, denominated as **maximum**. On the other hand, each of the importer countries do require a minimum quantity of oil to keep this agreement with Spain.

Our goal is to find the maximum amount of flow(tons of oil) that can circulate between countries in order to export the amount of olive oil required by each country.In this case, destination importers are **Ireland**, **Poland**,**Italy** and **Switzerland**.



!['oil'](img/oil.jpg)

### MODEL FORMULATION

#### SETS

- C = Set of Countries (each one represent a **node**). (to denote we use the subscript C.)
  
    - (**Destinations:** Ireland, Poland, Italy, Switzerland ; **In between:** France, Monaco, UK, Belgium, Netherlands, and Germany. **Origin**: Spain.)
  
- P = Set of connections (connected countries for flow). Shown as a set ($\text{country}_i , \text{country}_j$).




#### PARAMETERS

- Origin = source node, in our case **SPAIN**

- Maximum = maximum quantity of olive oil allowed to pass from country to country in pairs ($\text{country}_i \rightarrow \text{country}_j$) represented as **$M_{ij}$**.

- Needs = amount of olive oil required by importer countries. $N_{i}$.


#### VARIABLES

- $Flow_{ij}$ = flow between country *i* to country *j*
  
- For each pair of nodes $i,j \in C$ that are defined in P (as the explicit pairs allowed).
 
$$Flow_{ij}=\left\{\begin{array}{ll}
\text{max flow} & \textrm{if there is flow $j$ right after node $i$}\\
0 & \textrm{otherwise}
\end{array}\right.$$


#### OBJECTIVE FUNCTION

Objective function is to maximize the amount of olive oil arriving at the final nodes (importer countries $\in N$ ). Therefore, it is the total sum of the flow in every pair of countries **(connections)** that arrive to a final destination defined in the Needs Parameter (Ireland, Poland, Switzerland, Italy).

$$\text{max} \;\; \sum_{i,j\in P}^{} Flow_{ij}, \quad\forall j \in \text{Needs}$$


#### CONSTRAINTS

- 1. Flow of oil outward of each country (in each pair of connections in P) must be less than the maximum amount it has:

$$\forall{i,j \in C}, \;\text{for pairs ij} \in P,\quad  flow_{ij} \;\;\leq\;\; maximum_{ij}$$

- 2. **Flow Conservation Rule**. Flow into a node must be equal to the flow out of a node except if we are in source (Origin = Spain) or destination (Needs = Ireland, Switzerland, Italy, Poland), to fulfill.
  

$$ \forall{b \in C}, \sum_{i \in P} Flow_{ib} = \sum_{j \in P} Flow_{bj} \;\;\text{iff}\;\; b\neq j, \;j \in \text{Needs} \;\text{or} \;\;i \in \text{Origin} $$


- 3. We must ensure that that importer countries receive at least their required amount of oil pacted in the agreement.


$$\forall{b \in C}, \sum_{b,j \in C} Flow_{bj} \geq \text{Needs}_j$$





In [1]:
import pandas as pd
import numpy as np
from pyomo.environ import * 
import pandas
import math

In [2]:
%%writefile olive.dat
set C := Spain Ireland Poland Italy Switzerland France Monaco UK Belgium Netherlands Germany;
set P := (Spain, France) (Spain, Italy) (France, Monaco) (France, Italy) (France, Switzerland) (France, Belgium) (France, UK) (Belgium, Netherlands) (Belgium, Germany) (France, Germany) (UK, Ireland) (Netherlands, Germany) (Germany, Poland);

param origin := Spain;


param maximum :=
Spain France 160
Spain Italy 50
France Monaco 20
France Italy 150
France Switzerland 40
France Belgium 40
France UK 50
Belgium Netherlands 30
Belgium Germany 40
France Germany 30
UK Ireland 50
Netherlands Germany 20
Germany Poland 60;

param needs :=
Ireland 50
Poland 45
Italy 100
Switzerland 15;



Overwriting olive.dat


In [3]:
%%writefile olive_exp.py
from pyomo.environ import * 

model = AbstractModel()

model.C = Set()

# Initialize the Points with the cartesian product of the countries we have.
model.P = Set(within = model.C*model.C)

# Spain is the starting point -> ORIGIN NODE
model.origin = Param(within = model.C)

model.maximum = Param(model.P)

model.needs = Param(model.C)

# Decision variables
model.flow = Var(model.P, within=NonNegativeIntegers)

# Objective Function

def objective_rule(model):

    obj_val = sum(model.flow[origin,dest] for origin,dest in model.P if dest =='Ireland' or dest =='Poland' or dest == 'Italy' or dest =='Switzerland')

    return obj_val

model.obj = Objective(rule = objective_rule, sense = maximize)

# Constraint 1
def allowed_quantity(model, m, n):
    return model.flow[m, n] <= model.maximum[m, n]



# Constraint 2
def flow_rule(model , k):
    # if it is a final destination - dont flow out . If it is not then make sure the inflow and outflow are equal.
    if k in ["Italy", "Switzerland", "Ireland","Poland", value(model.origin)]:
        return Constraint.Skip
    
    # the inflow from one origin to a destination MUST equal the outflow from origin to destination.
    # calculating the inflow
    inflow = sum(model.flow[origin,dest] for (origin,dest) in model.P if dest==k)
    # calculating the outflow.
    outflow = sum(model.flow[origin,dest] for (origin,dest) in model.P if origin==k)

    return inflow==outflow


# Constraint 3
def requested_oil(model, i):
    if i in ["Italy", "Switzerland", "Ireland","Poland"]:
        return sum(model.flow[j, k] for (j,k) in model.P if k==i)>=model.needs[i]
    else:
        return Constraint.Skip


model.flow_rule = Constraint(model.C, rule = flow_rule)
model.allowed_quantity = Constraint(model.P, rule = allowed_quantity)
model.requested_oil = Constraint(model.C, rule = requested_oil)


Overwriting olive_exp.py


In [4]:
!pyomo solve  olive_exp.py  olive.dat --solver=glpk --summary 

!type results.yml

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.02] Creating model
[    0.04] Applying solver
[    0.26] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 210.0
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    flow : Size=13, Index=P
        Key                        : Lower : Value : Upper : Fixed : Stale : Domain
            ('Belgium', 'Germany') :     0 :  40.0 :  None : False : False : NonNegativeIntegers
        ('Belgium', 'Netherlands') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
             ('France', 'Belgium') :     0 :  40.0 :  None : False : False : NonNegativeIntegers
             ('France', 'Germany') :     0 :   5.0 :  None : False : False : NonNegativeIntegers
               ('France', 'Italy') :     0 :  50.0 :  None : False : False : NonNegativeIntegers
              ('France', 'Monac

#### ANALYSIS OF RESULTS

> Obtained flows

In [5]:
import yaml
with open('results.yml') as f:
    doc = yaml.load(f,Loader=yaml.FullLoader)
    l1 = doc["Solution"][1]["Variable"]
    l1 = list(l1.items())
    
for i in l1:
    print(i[0],':',i[1]["Value"])

flow[Belgium,Germany] : 40
flow[France,Belgium] : 40
flow[France,Germany] : 5
flow[France,Italy] : 50
flow[France,Switzerland] : 15
flow[France,UK] : 50
flow[Germany,Poland] : 45
flow[Spain,France] : 160
flow[Spain,Italy] : 50
flow[UK,Ireland] : 50


| Route | Max Flow solution | Maximum Allowed
| --- | --- | --- |
| Spain -> France | 160 | 160 |
| Spain -> Italy | 50 | 50 |
| France -> Belgium | 40 | 40 |
| France -> Germany | 5 | 30 |
| France -> Italy | 50 | 150 |
| France -> Switzerland | 15 | 40 |
| France -> UK | 50 | 50 |
| Belgium -> Germany | 40 | 40 |
| Germany -> Poland | 45 | 60 |
| UK -> Ireland | 50 | 50 |


As we can see in the previous table:

- 160 tons will pass from Spain to France
- 50 tons will pass from Spain to Italy
- 40 tons will pass from France to Belgium
- 5 tons will pass from France to Germany
- 50 tons will pass from France to Italy
- 15 tons will pass from France to Switzerland
- 50 tons will pass from France to UK
- 40 tons will pass from Belgium to Germany
- 45 tons will pass from Germany to Poland
- 50 tons will pass from UK to Ireland

## 2. Non-linear problem

In regional water planning, sources emitting pollutants might be required to
remove waste from the water system. Let $x_{j}$ be the **kg** of Biological Oxygen Demand (an often-used measure of pollution) to be removed at source *j*.

One model might be to **minimize** total costs to the region to meet specified pollution standards, 10 and 15 for the water quality points:



$$
\text{Minimize} \sum_{j=1}^{n} f_j(x_j)
$$


subject to a series of **constraints** described below:



!['water'](img/water.jpg)

### MODEL FORMULATION


#### SETS

- I = Water quality points, responsible of measuring pollution. Water system has 2 quality points which do the function of sensors.
- J = Source nodes at which pollution is removed. Water system has 3 nodes.


#### PARAMETERS



- $b_i$ = Minimum desired improvement in water quality at point *$i\in I$* in the
system.
- $a_{ij}$ = Quality response, at point *$i\in I$* in the water system, caused by removing
one kilogram of Biological Oxygen Demand at source *$j\in J$*.
- $u_j$ = Maximum kilograms of Biological Oxygen Demand that can be
removed at source *$j\in J$*.


#### VARIABLES
 - $x_{j}$,  *$j\in J$*: Quantity of Biological Oxygen Demand to be removed at node *j*


#### OBJECTIVE FUNCTION

- $f_j$ $(x_j)$ = Cost of removing $x_j$ kilograms of Biological Oxygen Demand at
source *$j\in J$*,

$$
\text{Minimize} \sum_{j=1}^{n} f_j(x_j)
$$



Where $f_j (x_j)$ and coefficients c,d and e have been defined as:



$$ f_j (x_j) = c\cdot x_{j}^{2} + d\cdot x_{j} + e$$


#### CONSTRAINTS


- Quality response multiplied by quantity of polution (in kg) must be higher than minimum desired improvement
  
  $$\sum_{j=1}^{n}  a_{ij} x_j ≥ b_i \quad i\in I$$

- Quantity of removed pollution (in kg) must be lower than maximum kg that can be removed at source j
 
  $$0 ≤ x_j ≤ u_j \quad j\in J$$

In [6]:
from scipy.optimize import minimize

# Parameters
n = 3  # Number of sources 
m = 2  # Number of water quality points

u = [15, 15, 20]  # Maximum kilograms of BOD that can be removed at each source
a = [[1, 2, 1], [0, 1, 2]]  # Quality response matrix: rows->(quality points) & cols->(source nodes)

b = [10, 15]  # Minimum desired improvement in water quality

c = [4, 8, 3]  # Quadratic coefficient for cost function
d = [2, 1.5, 3]  # Linear coefficient for cost function
e = [5, 3, 7]  # Constant term for cost function


In [7]:
# Define the cost function for removing x_j pounds of BOD at source j
def cost_function(x, c, d, e):
    return c * x**2 + d * x + e

In [8]:
# Define the objective function
def objective_function(x, c, d, e):
    return sum(cost_function(x[j], c[j], d[j], e[j]) for j in range(len(x)))

In [9]:
# Define the equality constraint functions
def equality_constraint(x, a, b):
    return [sum(a[i][j] * x[j] for j in range(len(x))) - b[i] for i in range(len(b))]

# Define the inequality constraint bounds
constraint_bounds = [(0, u[j]) for j in range(len(u))]

In [10]:
# Initial guess for x: initialize empty zeros vector
initial_guess = np.zeros(len(u))

In [19]:
#Solve the optimization problem
result = minimize(objective_function, initial_guess, args=(c, d, e),
                  constraints={'type': 'eq', 'fun': equality_constraint, 'args': (a, b)},
                  bounds=constraint_bounds, method='SLSQP')

# Display results
print("Optimal solution:")
print([round(x, 2) for x in result.x])
print("Optimal cost:")
print(round(result.fun, 2))

Optimal solution:
[0.15, 1.56, 6.72]
Optimal cost:
192.87


### INTERPRETATION

- At **Node 1** 0.15 kilograms of BOD are removed using the water system.
- At **Node 2** 1.56 kilograms of BOD are removed using the water system.
- At **Node 3** 6.72 kilograms of BOD are removed using the water system.

- Regarding the **optimal value**, we came up with a cost of 192.87€ to reduce pollution in this water system, meanwhile adhering to a minimum desired improvement in quality represented by $b_i$ for $i\in I$, [10, 15] respectively.

# Bibliography




https://oec.world/es/profile/bilateral-product/olive-oil-virgin/reporter/pol

For the **Non-linear problem** we got inspired from this reference:

https://web.mit.edu/15.053/www/AMP-Chapter-13.pdf
